In [1]:
from transformers import  AutoModelForCausalLM, AutoTokenizer,LlamaConfig, LlamaForCausalLM
import torch
import torch.nn as nn
import torch.nn.init as init
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%config InlineBackend.figure_format = 'retina'

### Code 2

In [2]:
# !pip install datasets -U

In [3]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset
import pandas as pd

In [4]:
df = pd.read_csv("output_1500.csv")

In [5]:
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
df["Text"] = df["Text"] + "<end_of_sen>"

In [7]:
df.iloc[0, 0]

'qui peut me renseigner sur cette maladie....symptomes,traitement,est ce grave?;va t-il pouvoir reprendre son boulot en agriculture ?<end_of_sen>'

In [8]:
df = df.dropna()

In [9]:
train_size = 700

In [10]:
df = df.iloc[:train_size, :]
df

,Text
0,qui peut me renseigner sur cette maladie....sy...
1,"Un des buts du fameux chemin spirituel, pour m..."
2,Peut-être qu'il faut le faire au moment du tél...
3,Le véhicule est propulsé par un moteur diesel ...
4,"Cela n’a rien à voir avec leur saveur, mais av..."
...,...
729,Entrez sur de nouveaux marchés avec nous !<end...
730,"Pour soutenir davantage la création, nous allo..."
731,veuillez m’excuser si je ne suis pas au bon en...
732,Ca me fait plaisir de voir des nouveaux qui éc...


In [11]:
tokenizer = AutoTokenizer.from_pretrained("test_tokenizer_1500", model_max_length = 64)

In [12]:
input_ids = tokenizer(df["Text"].to_list()[:], truncation = True, padding = True)["input_ids"]

In [13]:
token_list = []
for i in input_ids:
  token_list.extend(i)

In [14]:
len(token_list)

44800

In [15]:
df = pd.DataFrame(columns=["input_ids"])
df

,input_ids


In [16]:
context_len = 64      ## Taking less because I have less data
token_batch = []
for i in input_ids:
  token_batch.append(token_list[:context_len])
  token_list = token_list[context_len:]

In [17]:
len(token_batch[-1])

64

In [18]:
df["input_ids"] = token_batch
df

,input_ids
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1229, 5822, 1110, 1805, 17254, 197, 1078, 107..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
695,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
696,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1392, ..."
697,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
698,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
attn_mask = [[1]*64]*len(df)

In [20]:
# df["attention_mask"] = attn_mask
df['labels'] = df['input_ids']

In [21]:
# df = df.iloc[:100,:]
df.head()

,input_ids,labels
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1229, 5822, 1110, 1805, 17254, 197, 1078, 107...","[1229, 5822, 1110, 1805, 17254, 197, 1078, 107..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
hf_dataset = Dataset.from_pandas(df)
hf_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 700
})

In [23]:
split_dataset = hf_dataset.train_test_split(test_size=0.3)  # Adjust test_size as needed

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [24]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [25]:
train_dataset.to_parquet("fr_dataset_token_train.parquet")
eval_dataset.to_parquet("fr_dataset_token_test.parquet")

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

216720

### Code 1

In [26]:
tokenizer = AutoTokenizer.from_pretrained("test_tokenizer_1500")

In [27]:
len(tokenizer.vocab)

32769

In [28]:
config = LlamaConfig(hidden_size=512,
                     vocab_size=len(tokenizer.vocab),
                     num_attention_heads=16,
                     num_key_value_head =2,
                     num_hidden_layers=32,
                     intermediate_size=512,
                     eos_token_id = 2,
                     bos_token_id = 1,
                     max_position_embeddings=64)
config

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 32,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "max_position_embeddings": 64,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 16,
  "num_hidden_layers": 32,
  "num_key_value_head": 2,
  "num_key_value_heads": 16,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 32769
}

In [29]:
model_mis = LlamaForCausalLM(config)

In [30]:
for i,j in model_mis.named_parameters():
  if j.requires_grad and len(j.size()) > 1:
    init.xavier_uniform_(j.data)

In [31]:
total_param=0
for i,j in model_mis.named_parameters():
    total_param += j.numel()
print("Number of Parameters:",total_param/(10**6))

Number of Parameters: 92.308992


In [32]:
model_mis.save_pretrained("save_test_tokenizer")
tokenizer.save_pretrained("save_test_tokenizer")

('save_test_tokenizer\\tokenizer_config.json',
 'save_test_tokenizer\\special_tokens_map.json',
 'save_test_tokenizer\\tokenizer.json')

### Code 3

In [33]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import math

In [34]:
per_epoch = []
perplex = []

In [35]:
class PerplexityCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if state.log_history:  # Log history should have recent metrics
            for entry in reversed(state.log_history):
                if "eval_loss" in entry:
                    eval_loss = entry["eval_loss"]
                    perplexity = math.exp(eval_loss)
                    per_epoch.append(state.epoch)
                    perplex.append(perplexity)
                    if state.epoch % 10 == 0:  # Every 10 epochs
                        print(f"Epoch {state.epoch}")
                        print(f"Perplexity = {perplexity:.4f}")
                    break


In [36]:
training_args = TrainingArguments(
    output_dir="./fr_model",
    overwrite_output_dir=True,
    evaluation_strategy= 'epoch',
    logging_strategy= 'epoch',
    save_strategy='epoch',
    num_train_epochs=100,
    logging_steps=1,
    learning_rate=2e-3,
    bf16=True,
    do_train=True,
    per_device_train_batch_size= 100,
    save_steps=20,
    save_total_limit=2,
    report_to="none",
)

In [37]:
trainer = Trainer(
    model=model_mis,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks= [PerplexityCallback()]
)

In [38]:
tokenizer.pad_token = tokenizer.eos_token

In [39]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 7.8823, 'grad_norm': 10.83006763458252, 'learning_rate': 0.00198, 'epoch': 1.0}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 5.341874122619629, 'eval_runtime': 65.3044, 'eval_samples_per_second': 3.216, 'eval_steps_per_second': 0.413, 'epoch': 1.0}
{'loss': 4.5232, 'grad_norm': 4.135104656219482, 'learning_rate': 0.00196, 'epoch': 2.0}


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 6.210764408111572, 'eval_runtime': 33.8903, 'eval_samples_per_second': 6.196, 'eval_steps_per_second': 0.797, 'epoch': 99.0}
Epoch 100.0
Perplexity = 498.0818
{'loss': 0.1398, 'grad_norm': 0.05096082389354706, 'learning_rate': 0.0, 'epoch': 100.0}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 6.210933208465576, 'eval_runtime': 108.7882, 'eval_samples_per_second': 1.93, 'eval_steps_per_second': 0.248, 'epoch': 100.0}
{'train_runtime': 46055.7341, 'train_samples_per_second': 1.064, 'train_steps_per_second': 0.011, 'train_loss': 1.2263152315616608, 'epoch': 100.0}


TrainOutput(global_step=500, training_loss=1.2263152315616608, metrics={'train_runtime': 46055.7341, 'train_samples_per_second': 1.064, 'train_steps_per_second': 0.011, 'total_flos': 1421196263424000.0, 'train_loss': 1.2263152315616608, 'epoch': 100.0})

In [40]:
perplex_df = pd.DataFrame({'Epochs': per_epoch, 'Perplexity': perplex})

In [41]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
perplex_df

,Epochs,Perplexity
0,2.0,208.903855
1,3.0,161.441705
2,4.0,160.420403
3,5.0,154.124282
4,6.0,108.961774
5,7.0,130.756497
6,8.0,125.037031
7,9.0,105.666495
8,10.0,115.096887
9,11.0,115.234450


In [42]:
custom_input = "La météo est magnifique cet après-midi."
input_dict = {'text': [custom_input]}

In [43]:
input_dict = {'input_ids': [tokenizer.encode(custom_input)]}
input_dict

{'input_ids': [[1278, 11133, 1110, 5185, 1889, 28342]]}

In [44]:
custom_dataset = Dataset.from_dict(input_dict)

In [45]:
predictions = trainer.predict(custom_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
generated_outputs = predictions.predictions  # This will be logits
generated_outputs.shape

(1, 6, 32769)

In [47]:
output_ids = torch.argmax(torch.tensor(generated_outputs), dim=2)
output_ids

tensor([[6628, 2105, 1213, 1201, 4788, 1180]])

In [48]:
custom_input = "Bonjour, comment allez-vous aujourd'hui ?"
tokenizer.decode(output_ids[0])

'conférenceautre i sont meilleurs avec'

In [49]:
output_ids

tensor([[6628, 2105, 1213, 1201, 4788, 1180]])

In [50]:
trainer.save_model("trained_model")
tokenizer.save_pretrained("trained_model")

('trained_model\\tokenizer_config.json',
 'trained_model\\special_tokens_map.json',
 'trained_model\\tokenizer.json')

In [51]:
def calculate_perplexity(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = math.exp(loss.item())

    return perplexity

# Test perplexity on custom prompts
test_prompts = [
"Bonjour, comment allez-vous aujourd'hui ?",
"Je suis très heureux de vous rencontrer.",
"La météo est magnifique cet après-midi.",
"Qu'est-ce que tu veux manger ce soir ?",
"Le voyage à Paris était incroyable.",
"Je travaille sur un projet de machine learning.",
"Le film que nous avons vu hier était génial.",
"As-tu entendu les dernières nouvelles ?",
"La science est un domaine fascinant.",
"Nous devons terminer notre tâche avant minuit."
]

# Calculate perplexity for each prompt
model = AutoModelForCausalLM.from_pretrained("trained_model")
tokenizer = AutoTokenizer.from_pretrained("trained_model")

for i, prompt in enumerate(test_prompts, 1):
    perplexity = calculate_perplexity(prompt, model, tokenizer)
    print(f"Prompt {i}: {prompt}")
    print(f"Perplexity: {perplexity:.2f}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt 1: Bonjour, comment allez-vous aujourd'hui ?
Perplexity: 11769310.06

Prompt 2: Je suis très heureux de vous rencontrer.
Perplexity: 494094.10

Prompt 3: La météo est magnifique cet après-midi.
Perplexity: 1105574.24

Prompt 4: Qu'est-ce que tu veux manger ce soir ?
Perplexity: 139059.49

Prompt 5: Le voyage à Paris était incroyable.
Perplexity: 547098.55

Prompt 6: Je travaille sur un projet de machine learning.
Perplexity: 242212.01

Prompt 7: Le film que nous avons vu hier était génial.
Perplexity: 6136713.17

Prompt 8: As-tu entendu les dernières nouvelles ?
Perplexity: 318060.91

Prompt 9: La science est un domaine fascinant.
Perplexity: 4303082.84

Prompt 10: Nous devons terminer notre tâche avant minuit.
Perplexity: 3917571.83

